In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import TargetEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR, LinearSVR
from sklearn.metrics import make_scorer, root_mean_squared_log_error, root_mean_squared_error
from tqdm import tqdm
from itertools import combinations
import matplotlib.pyplot as plt
import optuna
import xgboost as xgb
from sklearn.feature_selection import mutual_info_regression
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.neighbors import KNeighborsRegressor

from joblib import Parallel, delayed
import warnings
import time

In [2]:
IS_FEATURE_SELECTION_RUN = False

In [3]:
users = pd.read_csv("./data/external/train.csv")
users.columns = [x.lower() for x in users.columns]

assert users['id'].is_unique
# surprising!
assert users.notnull().all().all()

In [4]:
# competition's objective function in mind
users = users.assign(calories_log1p = lambda df_: np.log(df_['calories'] + 1))

In [ ]:
users.shape

In [ ]:
users.head()

In [ ]:
users.describe()

In [ ]:
users.dtypes

In [9]:
COLUMNS_TO_ONEHOT = ['sex']
onehot_encoder = OneHotEncoder(sparse_output=False, handle_unknown='warn').set_output(transform='pandas')
users_onehot = onehot_encoder.fit_transform(users[COLUMNS_TO_ONEHOT])

users = pd.concat([users, users_onehot], axis=1)

In [10]:
COLUMNS_CATEGORICAL = [
    'sex',
    ]

COLUMNS_NUMERIC = [
    'age',
    'height',
    'weight',
    'duration',
    'heart_rate',
    'body_temp',
    'sex_male',
    'sex_female'
    ]

In [11]:
# features_polynomial = [
#     'age',
#     'height',
#     'weight',
#     'duration',
#     'heart_rate',
#     'body_temp',
#     ]
# for x in features_polynomial:
#     users[f"{x}_sqrt"] = users[x]**(1/2)
#     users[f"{x}_sq"] = users[x]**2
#     COLUMNS_NUMERIC += [f"{x}_sqrt", f"{x}_sq"]

In [ ]:
FEATURES_INTERACTED = []

# adding 4th degree interactions (before polynomial transforms) failed to improve model
degrees_interaction = [2, 3]

for d in degrees_interaction:
    for features_interact in tqdm(list(combinations(COLUMNS_NUMERIC, d))):

        interaction_title = '_'.join(features_interact)
        FEATURES_INTERACTED.append(interaction_title)
        
        users[interaction_title] = users[list(features_interact)].prod(axis=1)

In [13]:
variances = users.select_dtypes(include='number').var()

columns_zero_variance = variances[variances == 0].index.tolist()

users = users.drop(columns=columns_zero_variance)

FEATURES_INTERACTED = list( set(FEATURES_INTERACTED).difference(set(columns_zero_variance)) )

In [14]:
# to manage features proliferation
# mutual_information = mutual_info_regression(
#     users[FEATURES_INTERACTED], 
#     users['calories_log1p'], 
#     n_jobs=-1
#     )
# mutual_information = (
#     pd.Series(mutual_information, index=FEATURES_INTERACTED)
#     .sort_values(ascending=False)
#     )
# features_low_importance = list(mutual_information[-1_000:].index)

# with open("./models/mutual_information__features_low_importance.txt", 'w') as file:
#     for item in features_low_importance:
#         file.write(f"{item}\n")

# with open("./models/mutual_information__features_low_importance.txt", 'r') as file:
#     features_low_importance = [line.strip() for line in file]

# users = users.drop(columns=features_low_importance)

# FEATURES_INTERACTED = list( set(FEATURES_INTERACTED) - set(features_low_importance) )

In [ ]:
# test main effects first
# -1.0 makes results decimals; 1 makes all zero ...
degrees_polynomial = [-1.0, 0.5, 2.0]
columns_polynomial = []
for x in COLUMNS_NUMERIC:
    for d in degrees_polynomial:
        series = users[x]**d
        if (all(~np.isinf(series))):
            users[f"{x}^{d}"] = series
            columns_polynomial += [f"{x}^{d}"]

In [ ]:
# following basis expansion via interactions, test:
    # new features set size
    # invariant (uninformative) features?
users.shape

In [17]:
XY = users

# Fit, Evaluate Baseline Model

In [18]:
# feature_transform_pipeline = ColumnTransformer(
#     [
#         ('target_encoder', TargetEncoder(target_type='continuous',cv=3), COLUMNS_CATEGORICAL),
#         ],
#     remainder='passthrough',
#     verbose_feature_names_out=False
# )

In [19]:
# what's faster: parallelize RF, and serial cv?
# or serial RF, parallel CV?

# in experiments, serial RF is faster.

# serial RF, parallel CV: 


# pipeline_e2e = Pipeline(
#     [
#         ('feature_transform_pipeline', feature_transform_pipeline),
#         ('model', RandomForestRegressor(n_estimators=100))
#      ]
# )

# preds_baseline = cross_val_predict(
#     pipeline_e2e,
#     XY[COLUMNS_CATEGORICAL + COLUMNS_NUMERIC],
#     XY['calories'],
#     n_jobs=-1,
#     cv=5,
#     verbose=2
# )

# score_baseline = root_mean_squared_log_error(XY['calories'], preds_baseline)

# score_baseline

In [20]:
# pipeline_e2e = Pipeline(
#     [
#         ('feature_transform_pipeline', feature_transform_pipeline),
#         ('model', RandomForestRegressor(n_estimators=100))
#      ]
# )

# scores = cross_val_score(
#     pipeline_e2e,
#     XY[COLUMNS_CATEGORICAL + COLUMNS_NUMERIC],
#     XY['calories_log1p'],
#     n_jobs=-1,
#     cv=5,
#     scoring='neg_root_mean_squared_error',
#     verbose=2
# )

# scores.mean(), scores

In [21]:
# long run time:
# summary score ≈ 0.06255 (from 2024-05-05 AM run)

# parallel CV, serial RF runtime > 30 min.
# hypothesis: as features set widens, then faster to parallel RF & serial cv.
# empirically, that works, runtime 16.5 min

# pipeline_e2e = Pipeline(
#     [
#         ('feature_transform_pipeline', feature_transform_pipeline),
#         ('model', RandomForestRegressor(n_estimators=100, n_jobs=-1))
#      ]
# )

# scores = cross_val_score(
#     pipeline_e2e,
#     XY[COLUMNS_CATEGORICAL + COLUMNS_NUMERIC + FEATURES_INTERACTED],
#     XY['calories_log1p'],
#     # n_jobs=-1,
#     cv=5,
#     scoring='neg_root_mean_squared_error',
#     verbose=2
# )

# scores.mean(), scores

# Feature Selection

In [35]:
def create_transformers_argument(features_model, features_universe_target_encode=COLUMNS_CATEGORICAL):

    transformers = []

    features_target_encode = [ftr for ftr in features_model if ftr in features_universe_target_encode]
    if features_target_encode:
        spec = ('target_encode', TargetEncoder(target_type='continuous',cv=3), features_target_encode)
        transformers.append(spec)

    return transformers

In [36]:
def fit_evaluate_model(X, y, n_jobs=1, model_title=None):
    """
    `model_title` helps with parallel calls to function over `X` datasets,
    and results need indexed.
    """

    transformers = create_transformers_argument(list(X.columns))
    feature_transform_pipeline = ColumnTransformer(
        transformers,
        remainder='passthrough',
        verbose_feature_names_out=False
    )
    pipeline_e2e = Pipeline(
        [
            ("transform_features", feature_transform_pipeline),
            ("standard_scale", StandardScaler()),
            ("model", Ridge(alpha=0.1))
        ]
    )

    scores = cross_val_score(
        pipeline_e2e,
        X,
        y,
        scoring='neg_root_mean_squared_error',
        cv=5,
        n_jobs=n_jobs
    )
    score_cv_summary = (-1 * scores.mean())

    if model_title:
        score_cv_summary = (model_title, score_cv_summary)

    return score_cv_summary

In [24]:
# it's strange to suppress a warning, but empirically and from research,
# haven't found the call-to-action from this warning.
# warnings.filterwarnings("ignore", message=".*A worker stopped while some jobs were given to the executor.*", category=UserWarning)

def fit_evaluate_marginal_model(
    feature_marginal, 
    X_challenger, 
    y, 
    n_jobs):

    features_challenger = list(X_challenger.columns)
    transformers = create_transformers_argument(features_challenger, COLUMNS_CATEGORICAL)
    feature_transform_pipeline = ColumnTransformer(transformers, remainder='passthrough', verbose_feature_names_out=False)
    pipeline_e2e = Pipeline(
        [
            ("transform_features", feature_transform_pipeline), 
            ("standard_scale", StandardScaler()),
            # ('to_float32', Float32Transformer()),
            # tested Support Vector Regression, but this was too time-consuming.
            # thought about KernelRidge too, but documentation suggests, equivalent to SVR
            # 'saga' chosen in attempt to resolve Runtime - divide by zero errors
            # TODO: might 'sag' be better fit for large data?
            ("model", Ridge(alpha=1.0, solver='saga'))
        ]
        )

    scores = cross_val_score(
        pipeline_e2e, 
        X_challenger, 
        y, 
        scoring='neg_root_mean_squared_error', 
        cv=5,  
        n_jobs=n_jobs
        )
    score_cv_summary = scores.mean()

    return (feature_marginal, -1 * score_cv_summary)

In [25]:
if IS_FEATURE_SELECTION_RUN:

    start = time.time()

    features_marginal = COLUMNS_CATEGORICAL + COLUMNS_NUMERIC + FEATURES_INTERACTED
    # using features universe, model search times practically infeasible
    # features_marginal = [x for x in features_marginal if x not in features_low_importance]
    features_champion = []
    features_added_count = 0

    # dummy, to kick off procedure
    champion_score = 10_000
    challenger_score = 1_000
    champions_score_sequence = [champion_score]

    # when challenger improves upon champion, continue extending challenger.
    # when challenger loses to champion, challenger has become too complex.
    while champion_score >= challenger_score:

        challengers_scores = Parallel(n_jobs=12)(
            delayed(fit_evaluate_marginal_model)(x, XY[features_champion + [x]], XY['calories_log1p'], 1) 
            for x in features_marginal
        )

        feature_marginal_challenger, challenger_score = min(challengers_scores, key=lambda x: x[1])

        print(f"Challenger score: {challenger_score}")
        print(f"Champion score: {champion_score}")
        elapsed = time.time() - start
        print("Cell run time:", time.strftime("%H:%M:%S", time.gmtime(elapsed)))

        # if challenger improves upon champion, then replace champion with challenger.
        if challenger_score < champion_score:

            features_champion += [feature_marginal_challenger]
            champion_score = challenger_score
            features_marginal.remove(feature_marginal_challenger)
            print(f"{feature_marginal_challenger} selected in this step.")

            features_added_count += 1
            print(f"Model features count comes to {features_added_count}.")

        champions_score_sequence.append(champion_score)

    features_champion

In [26]:
# imported from GPU-powered runs

FEATURES_SELECTED_RIDGE_DEFAULT_PENALTY = [
    'body_temp',
    'heart_rate',
    'age_duration',
    'age_duration_body_temp',
    'duration',
    'duration_heart_rate_body_temp',
    'duration_body_temp',
    'age_body_temp',
    'sex_male',
    'age_heart_rate_sex_female',
    'height_heart_rate_sex_male',
    'weight_body_temp_sex_female',
    'age_heart_rate_body_temp',
    'age_body_temp_sex_female',
    'heart_rate_sex_female',
    'height_weight_sex_male'
]

FEATURES_SELECTED_RIDGE_ALPHA_0P1 = [
    'body_temp',
    'heart_rate',
    'age_duration',
    'age_duration_body_temp',
    'duration',
    'duration_heart_rate_body_temp',
    'duration_body_temp',
    'duration_heart_rate',
    'age',
    'sex_female',
    'age_heart_rate_sex_female',
    'height_heart_rate_sex_male',
    'weight_body_temp_sex_female',
    'age_heart_rate',
    'age_body_temp_sex_male',
    'heart_rate_sex_male',
    'height_weight_sex_male',
    'duration_heart_rate_sex_male',
    'duration_heart_rate_sex_female',
    'duration_sex_male',
    'weight_sex_female',
    'duration_sex_female',
    'age_duration_heart_rate',
    'age_heart_rate_body_temp',
    'age_body_temp',
    'height_body_temp_sex_male',
    'age_weight_sex_female',
    'duration_body_temp_sex_female',
    'duration_body_temp_sex_male',
    'age_weight_heart_rate',
    'height_sex_female',
    'age_height_heart_rate',
    'height_duration_sex_male',
    'heart_rate_body_temp_sex_female',
    'weight_heart_rate_sex_female',
    'age_weight_body_temp',
    'weight_body_temp_sex_male',
    'age_height_sex_female',
    'age_weight',
    'weight_duration_body_temp',
    'weight_duration',
    'weight_duration_heart_rate',
    'age_sex_male',
    'weight_heart_rate_sex_male',
    'weight_heart_rate_body_temp',
    'age_height_body_temp',
    'heart_rate_body_temp_sex_male',
    'heart_rate_sex_female',
    'weight_heart_rate',
    'height_heart_rate_sex_female',
    'weight',
    'height_body_temp',
    'age_duration_sex_male',
    'heart_rate_body_temp',
    'weight_body_temp',
    'height_weight',
    'age_height_weight',
    'height_weight_heart_rate',
    'height_heart_rate_body_temp',
    'sex_male',
    'height_body_temp_sex_female',
    'weight_sex_male',
    'height_duration_heart_rate',
    'height_duration_sex_female',
    'height_duration_body_temp',
    'height_duration',
    'age_body_temp_sex_female',
    'weight_duration_sex_female'
    ]

FEATURES_SELECTED_POLYNOMIALS_RIDGE_ALPHA_0P1 = [
    'duration^0.5',
    'duration',
    'heart_rate^-1.0',
    'age^0.5',
    'duration^2.0',
    'sex_female^0.5',
    'age_heart_rate_sex_female',
    'height_heart_rate_sex_male',
    'age_weight_duration',
    'age_body_temp_sex_male',
    'age_height_heart_rate',
    'age_height_body_temp',
    'duration^-1.0',
    'heart_rate_sex_female',
    'age_weight_sex_male',
    'age_heart_rate_sex_male',
    'duration_heart_rate_sex_female',
    'duration_body_temp_sex_female',
    'body_temp_sex_female',
    'weight_sex_female',
    'weight_heart_rate',
    # 'weight_duration_sex_male',
    # 'age_sex_male',
    # 'age_height_weight',
    # 'duration_sex_male',
    # 'weight_heart_rate_sex_female',
    # 'age_body_temp_sex_female',
    # 'heart_rate_body_temp_sex_female',
    # 'weight_duration_heart_rate',
    # 'heart_rate^0.5'
    ]

FEATURES_SELECTED_SVR_C_0P1 = [
    'body_temp',
    'heart_rate',
    'age_duration',
    'age_duration_body_temp',
    'duration',
    'duration_heart_rate_body_temp',
    'age_weight_duration',
    'weight',
    'age_height_weight',
    'duration_body_temp',
    'weight_heart_rate_body_temp',
    'age_height_heart_rate',
    'sex_female',
    'age_duration_sex_female',
    'weight_heart_rate_sex_female',
    'age_sex_male',
    'age_weight_sex_female',
    'age_height',
    'heart_rate_body_temp_sex_male',
    'weight_body_temp'
    ]

FEATURES_SELECTED_SVR = [
    'body_temp',
    'heart_rate',
    'age_duration',
    'age_duration_body_temp',
    'duration',
    'duration_heart_rate_body_temp',
    'age_weight_duration',
    'weight_body_temp',
    'age_height_weight',
    'duration_body_temp',
    'weight_heart_rate',
    'sex_female',
    'height_heart_rate_sex_female',
    'age_sex_male',
    'weight',
    'height_duration_sex_female'
]

FEATURES_SELECTED_SVR_DEGREE4 = [
    'body_temp',
    'heart_rate',
    'age_duration',
    'age_duration_body_temp',
    'duration',
    'duration_heart_rate_body_temp',
    'age_weight_duration_heart_rate',
    'sex_male',
    'age_heart_rate_body_temp_sex_male',
    'height_heart_rate_body_temp_sex_male',
    'duration_body_temp',
    'age_weight',
    'age_height_weight_heart_rate',
    'weight_heart_rate_sex_female',
    'age_sex_male',
    'heart_rate_body_temp_sex_female'
]

# FEATURES_SELECTED_SVR_DEGREES_2_3_INTERACTIONS = [
#     'duration_sqrt',
#     'duration',
#     'age_sqrt_duration_sqrt_heart_rate_sq',
#     'age_duration_heart_rate_sq',
#     'age_sq_duration_sq_body_temp_sqrt',
#     'height_sqrt_duration_sqrt_duration_sq',
#     'sex_male',
#     'age_weight_sqrt_duration_sq',
#     'duration_age_sq_heart_rate_sq',
#     'age_heart_rate_duration_sq',
#     'age_age_sq_duration_sq',
#     'body_temp_heart_rate_sqrt',
#     'heart_rate_body_temp_sq',
#     'heart_rate_duration_sq_body_temp_sq',
#     'duration_age_sqrt_duration_sq',
#     'age',
#     'body_temp_heart_rate_sqrt_body_temp_sq',
#     'duration_sqrt_duration_sq_body_temp_sqrt',
#     'heart_rate_height_sq_duration_sq',
#     'age_sq_duration_sqrt_duration_sq'
#     ]

In [27]:
# FEATURES_SELECTED = list( 
#     set(FEATURES_SELECTED_RIDGE) 
#     | set(FEATURES_SELECTED_SVR) 
#     | set(FEATURES_SELECTED_SVR_DEGREE4)
#     )

# FEATURES_SELECTED = FEATURES_SELECTED_POLYNOMIALS_RIDGE_ALPHA_0P1

# Fit Models

## Individual Contributors (IC)

In [28]:
# def objective(trial):

#     params = {
#         "n_estimators": 1_000,
#         "objective": "reg:squarederror",
#         "booster": "gbtree",
#         # approximately sorted on hyperparam influence, descending
#         "eta": trial.suggest_float("eta", 1e-2, 0.1, log=True),
#         "max_depth": trial.suggest_int("max_depth", 1, 21, step=2),
#         "subsample": trial.suggest_float("subsample", 0.4, 1.0),
#         "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
#         "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
#         "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
#         "gamma": trial.suggest_float("gamma", 1e-8, 1.0, log=True),
#         }

#     scores = cross_val_score(
#         xgb.XGBRegressor(**params), 

#         XY[FEATURES_SELECTED],

#         XY['calories_log1p'], 

#         # as optuna _minimizes_, beware initially negative result
#         scoring='neg_root_mean_squared_error', 

#         cv=5
#         )
#     score_cv_summary = scores.mean()

#     return -1 * score_cv_summary

# study = optuna.create_study()

# study.optimize(
#     objective,
#     # as hyperparameters set size increases, likely need more trials
#     n_trials=35,
#     # if optuna returns nulls in y_pred, don't fail the entire study
#     catch=(ValueError,),
#     n_jobs=-1,
#     timeout=1 * 60 * 60,
# )

# study.best_params

In [29]:
# expect ~0.6 for any individual model

CHAMPION_PARAMS_SVR_0P1_FEATURES = {
    "n_estimators": 1_000,
    "objective": "reg:squarederror",
    "booster": "gbtree",
    'eta': 0.017685285832108887,
    'max_depth': 7,
    'subsample': 0.9081335480436096,
    'colsample_bytree': 0.7181715686597967,
    'lambda': 0.0031679606274101833,
    'alpha': 1.3956904209433707e-07,
    'gamma': 7.710204038839976e-05
 }

CHAMPION_PARAMS_HYBRID_FEATURES = {
    "n_estimators": 1_000,
    "objective": "reg:squarederror",
    "booster": "gbtree",
    'eta': 0.021941020992632378,
    'max_depth': 7,
    'subsample': 0.8180872703144669,
    'colsample_bytree': 0.642916966044508,
    'lambda': 3.24525375145767e-07,
    'alpha': 0.9854875114734817,
    'gamma': 3.62461567739456e-05
 }

CHAMPION_PARAMS_RIDGE_0P1_FEATURES = {
    "n_estimators": 1_000,
    "objective": "reg:squarederror",
    "booster": "gbtree",
    'eta': 0.02497863296780753,
    'max_depth': 15,
    'subsample': 0.9927172119082496,
    'colsample_bytree': 0.22427335166711926,
    'lambda': 9.812890689973725e-06,
    'alpha': 0.4386722082854982,
    'gamma': 0.01035902889930934
    }

CHAMPION_PARAMS_POLYNOMIALS_RIDGE_0P1 = {
    "n_estimators": 1_000,
    "objective": "reg:squarederror",
    "booster": "gbtree",
    'eta': 0.012344853649482653,
    'max_depth': 9,
    'subsample': 0.6835261426364755,
    'colsample_bytree': 0.898511409057206,
    'lambda': 0.0008844118731907166,
    'alpha': 5.021206527308231e-08,
    'gamma': 4.558181241103821e-07
    }

# expect ~0.06 score
CHAMPION_PARAMS_RIDGE_V1_FEATURES = {
    'eta': 0.009447724417129216,
    'max_depth': 9,
    'subsample': 0.7975742692448574,
    'colsample_bytree': 0.46241012463825576,
    'lambda': 1.0805178355851031e-06,
    'alpha': 0.9133372453796443,
    'gamma': 0.00396569352508906
    }

In [30]:
# scores = cross_val_score(
#     xgb.XGBRegressor(**CHAMPION_PARAMS_POLYNOMIALS_RIDGE_0P1), 
#     XY[FEATURES_SELECTED_SVR_C_0P1],
#     XY['calories_log1p'], 
#     # as optuna _minimizes_, beware initially negative result
#     scoring='neg_root_mean_squared_error', 
#     cv=5
#     )

# scores.mean(), scores

In [31]:
# def objective(trial):

#     params = {
#         "n_neighbors": trial.suggest_int("n_neighbors", 5, 55, step=10),
#         "weights": trial.suggest_categorical("weights", ['uniform', 'distance'])
#         }

#     scores = cross_val_score(
#         Pipeline([ ('standardize', StandardScaler()), ('model', KNeighborsRegressor(**params)) ]), 
#         XY[FEATURES_SELECTED_RIDGE_DEFAULT_PENALTY],
#         XY['calories_log1p'], 
#         # as optuna _minimizes_, beware initially negative result
#         scoring='neg_root_mean_squared_error', 
#         cv=5
#         )
#     score_cv_summary = scores.mean()

#     return -1 * score_cv_summary

# study = optuna.create_study()

# study.optimize(
#     objective,
#     n_trials=20,
#     # if optuna returns nulls in y_pred, don't fail the entire study
#     catch=(ValueError,),
#     n_jobs=-1,
#     timeout=1 * 60 * 60,
# )

# study.best_params

## Ensemble

In [32]:
class ModelEnsemble(BaseEstimator, RegressorMixin):
    """
    Aggregate several lower-level models.
    """    
    def __init__(self):
        self.models = {}

    def fit_components(self, X, y):
        """
        Ensemble model learns from component models' out-of-fold predictions,
        otherwise expect overfitting.
        Persist models' predictions to allow a separate, dedicated step 
        to fit-evaluate ensemble blender.

        Model experimentation notes:
        - Expect radial basis function kernel fit time is prohibitive with data of ~750K obs.
        - Insignificant benefit from adding KNN 
        - Insignficant benefit from adding random forest
        """

        XY_ensemble = y.to_frame('y')

        preds_svr_default__xgb = cross_val_predict(
            xgb.XGBRegressor(**CHAMPION_PARAMS_HYBRID_FEATURES), 
            X[FEATURES_SELECTED_SVR],
            y, 
            cv=5,
            verbose=1
            )
        svr_default__xgb = xgb.XGBRegressor(**CHAMPION_PARAMS_HYBRID_FEATURES)
        svr_default__xgb.fit(X[FEATURES_SELECTED_SVR], y)
        self.models['svr_default__xgb'] = svr_default__xgb
        XY_ensemble = XY_ensemble.assign(svr_default__xgb = preds_svr_default__xgb)
        print("SVR (default penalty) feature selection - xgboost fit.")
        
        preds_svr_0p1 = cross_val_predict(
            xgb.XGBRegressor(**CHAMPION_PARAMS_HYBRID_FEATURES), 
            X[FEATURES_SELECTED_SVR_C_0P1],
            y, 
            cv=5,
            verbose=1
            )
        svr_0p1__xgb = xgb.XGBRegressor(**CHAMPION_PARAMS_HYBRID_FEATURES)
        svr_0p1__xgb.fit(X[FEATURES_SELECTED_SVR_C_0P1], y)
        self.models['svr_0p1__xgb'] = svr_0p1__xgb
        XY_ensemble = XY_ensemble.assign(svr_0p1__xgb = preds_svr_0p1)
        print("SVR (below-default penalty) feature selection - xgboost fit.")

        preds_ridge_default__xgb = cross_val_predict(
            xgb.XGBRegressor(**CHAMPION_PARAMS_RIDGE_V1_FEATURES), 
            X[FEATURES_SELECTED_RIDGE_DEFAULT_PENALTY],
            y, 
            cv=5,
            verbose=1
            )
        ridge_default__xgb = xgb.XGBRegressor(**CHAMPION_PARAMS_RIDGE_V1_FEATURES)
        ridge_default__xgb.fit(X[FEATURES_SELECTED_RIDGE_DEFAULT_PENALTY], y)
        self.models['ridge_default__xgb'] = ridge_default__xgb
        XY_ensemble = XY_ensemble.assign(ridge_default__xgb = preds_ridge_default__xgb)
        print("Ridge (default penalty) feature selection - xgboost fit.")

        preds_ridge_0p1 = cross_val_predict(
            xgb.XGBRegressor(**CHAMPION_PARAMS_RIDGE_0P1_FEATURES), 
            X[FEATURES_SELECTED_RIDGE_ALPHA_0P1],
            y, 
            cv=5,
            verbose=1
            )
        ridge_0p1__xgb = xgb.XGBRegressor(**CHAMPION_PARAMS_RIDGE_0P1_FEATURES)
        ridge_0p1__xgb.fit(X[FEATURES_SELECTED_RIDGE_ALPHA_0P1], y)
        self.models['ridge_0p1__xgb'] = ridge_0p1__xgb
        XY_ensemble = XY_ensemble.assign(ridge_0p1__xgb = preds_ridge_0p1)
        print("Ridge (below-default penalty) feature selection - xgboost fit.")

        preds_polynomials_ridge_0p1 = cross_val_predict(
            xgb.XGBRegressor(**CHAMPION_PARAMS_POLYNOMIALS_RIDGE_0P1), 
            X[FEATURES_SELECTED_POLYNOMIALS_RIDGE_ALPHA_0P1],
            y, 
            cv=5,
            verbose=1
            )
        ridge_polynomials_0p1__xgb = xgb.XGBRegressor(**CHAMPION_PARAMS_POLYNOMIALS_RIDGE_0P1)
        ridge_polynomials_0p1__xgb.fit(X[FEATURES_SELECTED_POLYNOMIALS_RIDGE_ALPHA_0P1], y)
        self.models['ridge_polynomials_0p1__xgb'] = ridge_polynomials_0p1__xgb
        XY_ensemble = XY_ensemble.assign(ridge_polynomials_0p1__xgb = preds_polynomials_ridge_0p1)
        print("Ridge (below-default penalty) polynomials feature selection - xgboost fit.")

        preds_ridge_0p1__ridge = cross_val_predict(
            Pipeline([ ('standardize', StandardScaler()), ('model', Ridge(alpha=0.1)) ]), 
            X[FEATURES_SELECTED_RIDGE_ALPHA_0P1],
            y, 
            cv=5,
            verbose=1
            )
        ridge_0p1__ridge = Ridge(alpha=0.1)
        ridge_0p1__ridge.fit(X[FEATURES_SELECTED_RIDGE_ALPHA_0P1], y)
        self.models['ridge_0p1__ridge'] = ridge_0p1__ridge
        XY_ensemble = XY_ensemble.assign(ridge_0p1__ridge = preds_ridge_0p1__ridge)
        print("Ridge (below-default penalty) feature selection - Ridge fit.")

        # tested, but public LB performance declined
        # preds_ridge_polynomials_0p1__ridge = cross_val_predict(
        #     Pipeline([ ('standardize', StandardScaler()), ('model', Ridge(alpha=0.1)) ]), 
        #     X[FEATURES_SELECTED_POLYNOMIALS_RIDGE_ALPHA_0P1],
        #     y, 
        #     cv=5,
        #     verbose=1
        #     )
        # ridge_polynomials_0p1__ridge = Ridge(alpha=0.1)
        # ridge_polynomials_0p1__ridge.fit(X[FEATURES_SELECTED_POLYNOMIALS_RIDGE_ALPHA_0P1], y)
        # self.models['ridge_polynomials_0p1__ridge'] = ridge_polynomials_0p1__ridge
        # XY_ensemble = XY_ensemble.assign(ridge_polynomials_0p1__ridge = preds_ridge_polynomials_0p1__ridge)
        # print("Ridge (below-default penalty) polynomials feature selection - Ridge fit.")
        
        self.XY_ensemble_fit = XY_ensemble

    def fit(self, X, y):

        self.fit_components(X, y)

        blender_pipeline_e2e = Pipeline(
            [
                ("standard_scale", StandardScaler()),
                # taken from CV step
                ("model", Ridge(alpha=30))
            ]
            )
        blender_pipeline_e2e.fit(
            self.XY_ensemble_fit.drop(columns='y'),
            self.XY_ensemble_fit['y']
            )
        self.blender = blender_pipeline_e2e

    def predict(self, X):

        X_models = []
        for title, model in self.models.items():
            predictions = model.predict(X[model.feature_names_in_])
            predictions = pd.Series(predictions, name=title)
            X_models.append(predictions)

        X_models = pd.concat(X_models, axis=1)

        preds = self.blender.predict(X_models)

        return preds

In [ ]:
model_ensemble = ModelEnsemble()
model_ensemble.fit_components(XY.drop(columns='calories_log1p'), XY['calories_log1p'])

In [ ]:
start = time.time()

features_marginal = list(model_ensemble.XY_ensemble_fit.drop(columns='y'))
# using features universe, model search times practically infeasible
# features_marginal = [x for x in features_marginal if x not in features_low_importance]
features_champion = []
features_added_count = 0

# dummy, to kick off procedure
champion_score = 10_000
challenger_score = 1_000
champions_score_sequence = [champion_score]

# when challenger improves upon champion, continue extending challenger.
# when challenger loses to champion, challenger has become too complex.
while champion_score >= challenger_score:

    challengers_scores = Parallel(n_jobs=12)(
        delayed(fit_evaluate_model)(
            model_ensemble.XY_ensemble_fit[features_champion + [x]], 
            model_ensemble.XY_ensemble_fit['y'], 
            model_title=x
            )
        for x in features_marginal
    )

    feature_marginal_challenger, challenger_score = min(challengers_scores, key=lambda x: x[1])

    print(f"Challenger score: {challenger_score}")
    print(f"Champion score: {champion_score}")
    elapsed = time.time() - start
    print("Cell run time:", time.strftime("%H:%M:%S", time.gmtime(elapsed)))

    # if challenger improves upon champion, then replace champion with challenger.
    if challenger_score < champion_score:

        features_champion += [feature_marginal_challenger]
        champion_score = challenger_score
        features_marginal.remove(feature_marginal_challenger)
        print(f"{feature_marginal_challenger} selected in this step.")

        features_added_count += 1
        print(f"Model features count comes to {features_added_count}.")

    champions_score_sequence.append(champion_score)

features_champion

In [ ]:
XY_ensemble = model_ensemble.XY_ensemble_fit.copy()

def objective(trial):
    
    parameters = {'alpha': trial.suggest_float("alpha", 1e-1, 1_000, log=True)}

    pipeline_e2e = Pipeline(
        [
            ("standard_scale", StandardScaler()),
            ("model", Ridge(**parameters))
        ]
        )

    scores = cross_val_score(
        pipeline_e2e, 
        XY_ensemble.drop(columns='y'),
        XY_ensemble['y'], 
        # as optuna _minimizes_, beware initially negative result
        scoring='neg_root_mean_squared_error', 
        cv=5
        )
    score_cv_summary = scores.mean()

    return -1 * score_cv_summary

study = optuna.create_study()

study.optimize(
    objective,
    # as hyperparameters set size increases, likely need more trials
    n_trials=35,
    # if optuna returns nulls in y_pred, don't fail the entire study
    catch=(ValueError,),
    n_jobs=-1,
    timeout=1 * 60 * 60,
)

study.best_params

In [44]:
# does SVR ensembler deliver better results?
# rbf kernel permits interactions. but with 750K observations, seems infeasible.
# LinearSVR should scale better to large data, versus SVR with kernel='linear'.
# in experiments, SVR ensembler performs ~.001 worse versus Ridge.

# XY_ensemble = model_ensemble.XY_ensemble_fit.copy()

# def objective(trial):
    
#     parameters = {
#         'C': trial.suggest_float("alpha", 1e-2, 100, log=True)
#         }

#     pipeline_e2e = Pipeline(
#         [
#             ("standard_scale", StandardScaler()),
#             ("model", LinearSVR(**parameters))
#         ]
#         )

#     scores = cross_val_score(
#         pipeline_e2e, 
#         XY_ensemble.drop(columns='y'),
#         XY_ensemble['y'], 
#         # as optuna _minimizes_, beware initially negative result
#         scoring='neg_root_mean_squared_error', 
#         cv=5
#         )
#     score_cv_summary = scores.mean()

#     return -1 * score_cv_summary

# study = optuna.create_study()

# study.optimize(
#     objective,
#     # as hyperparameters set size increases, likely need more trials
#     n_trials=10,
#     # if optuna returns nulls in y_pred, don't fail the entire study
#     catch=(ValueError,),
#     n_jobs=-1,
#     timeout=1 * 60 * 60,
# )

# study.best_params

In [50]:
# with ensemble's ~4 component models,
# gradient boosting machine is a lower-performing blender (~0.6 score).
# Implies hypothesis -> prediction components are relatively strong signals.

# def objective(trial):

#     params = {
#         "n_estimators": 1_000,
#         "objective": "reg:squarederror",
#         "booster": "gbtree",
#         # approximately sorted on hyperparam influence, descending
#         "eta": trial.suggest_float("eta", 1e-2, 0.1, log=True),
#         "max_depth": trial.suggest_int("max_depth", 1, 21, step=2),
#         "subsample": trial.suggest_float("subsample", 0.4, 1.0),
#         "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
#         "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
#         "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
#         "gamma": trial.suggest_float("gamma", 1e-8, 1.0, log=True),
#         }

#     scores = cross_val_score(
#         xgb.XGBRegressor(**params), 
#         XY_ensemble.drop(columns='y'),
#         XY_ensemble['y'], 
#         scoring='neg_root_mean_squared_error', 
#         cv=5
#         )
#     score_cv_summary = scores.mean()

#     return -1 * score_cv_summary

# study = optuna.create_study()

# study.optimize(
#     objective,
#     # as hyperparameters set size increases, likely need more trials
#     n_trials=35,
#     # if optuna returns nulls in y_pred, don't fail the entire study
#     catch=(ValueError,),
#     n_jobs=-1,
#     timeout=1 * 60 * 60,
# )

# study.best_params

In [51]:
# interesting, Random Forest also a poor blender ... too complex?
# scores = cross_val_score(
#     RandomForestRegressor(), 
#     XY_ensemble.drop(columns='y'),
#     XY_ensemble['y'], 
#     scoring='neg_root_mean_squared_error', 
#     cv=5,
#     n_jobs=-1
#     )
# score_cv_summary = scores.mean()

# score_cv_summary, scores

In [ ]:
MODEL_ENSEMBLE = ModelEnsemble()
MODEL_ENSEMBLE.fit(XY.drop(columns='calories_log1p'), XY['calories_log1p'])

In [ ]:
break

# TODO: Neural Nets

Motivation: Kuhn's Neural Net model performance, with noise features screened

In [37]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import autokeras as ak
import keras

In [41]:
from tensorflow.keras.layers import BatchNormalization, Dropout, Input, Activation
from keras import Model

def build_model(size):

    x_in = Input(shape=(size,))
    x = Dense(512)(x_in)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.3)(x)

    x = Dense(256)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.3)(x)

    x = Dense(128, activation='relu')(x)
    x = Dense(1, activation='linear')(x)

    model = Model(inputs=x_in, outputs=x)
    
    return model

In [ ]:
X = XY_ensemble.drop(columns='y')
y = XY_ensemble['y']

scores_cv = []
kf = KFold(n_splits=5, shuffle=True, random_state=42)
for indexes_train, indexes_test in kf.split(X):
    
    X_train, X_test = X.loc[indexes_train], X.loc[indexes_test]
    
    y_train, y_test = y.loc[indexes_train], y.loc[indexes_test]

    features_mean = X_train.mean()
    features_std = X_train.std()
    X_train_trfm = (X_train - features_mean) / features_std

    # model = Sequential([
    #     Dense(128, activation='relu', input_shape=(X_train_trfm.shape[1],)),
    #     Dense(64, activation='relu'),
    #     Dense(1)  # Output layer for regression
    # ])
    # https://www.kaggle.com/code/garrickchinnis/calorie-expenditure-keras
    # TODO: how does this notebook achieve a good score?
    model = keras.Sequential([
        Dense(128, activation='relu', input_shape=[X_train_trfm.shape[1]]),
        Dense(64, activation='relu'),
        Dense(64, activation='relu'),
        Dense(1)])
    model = build_model(X_train_trfm.shape[1])

    model.compile(optimizer='adam', loss='mse', metrics=['mse'])
    # model = ak.TabularRegressor(max_trials=10, loss='mse')
    # model.fit(X_train_trfm, y_train, epochs=50)

    history = model.fit(X_train_trfm, y_train, epochs=4, batch_size=512)
    
    X_test_trfm = (X_test - features_mean) / features_std
    # test_loss = np.sqrt(model.evaluate(X_test_trfm, y_test))
    preds = model.predict(X_test_trfm)
    test_loss = root_mean_squared_error(y_test, preds)
    print(f"CV RMSE: {test_loss}")

    scores_cv.append(test_loss)

# keras `history` abstraction presents training set loss.
# expect far-optimistic evaluation metric ...

# TODO: cross_val_score equivalent for keras workflow?

In [ ]:
break

# Data Exploration Report

In [ ]:
users.plot.scatter('weight_duration_heart_rate', 'calories');

In [ ]:
users.plot.scatter('duration', 'calories_log1p');

In [ ]:
users.plot.scatter('age_duration', 'calories');

In [ ]:
users.plot.scatter('duration_sex_female', 'calories');

In [ ]:
users.plot.scatter('age', 'calories');

In [ ]:
users.plot.scatter('height', 'calories');

In [ ]:
users.plot.scatter('weight', 'calories');

In [ ]:
# the competition's objective function differs from MSE -- RF trains on MSE.

# Deploy

In [54]:
users_test = pd.read_csv("./data/external/test.csv")
users_test.columns = [x.lower() for x in users_test.columns]

users_test_onehot = onehot_encoder.transform(users_test[COLUMNS_TO_ONEHOT])

users_test = pd.concat([users_test, users_test_onehot], axis=1)

In [ ]:
for d in degrees_interaction:
    for features_interact in tqdm(list(combinations(COLUMNS_NUMERIC, d))):

        interaction_title = '_'.join(features_interact)
        FEATURES_INTERACTED.append(interaction_title)
        
        users_test[interaction_title] = users_test[list(features_interact)].prod(axis=1)

In [ ]:
# test main effects first
# -1.0 makes results decimals; 1 makes all zero ...
degrees_polynomial = [-1.0, 0.5, 2.0]
columns_polynomial = []
for x in COLUMNS_NUMERIC:
    for d in degrees_polynomial:
        series = users_test[x]**d
        if (all(~np.isinf(series))):
            users_test[f"{x}^{d}"] = series
            columns_polynomial += [f"{x}^{d}"]

In [ ]:
yhat_log1p_test = MODEL_ENSEMBLE.predict(users_test)
yhat_test = np.exp(yhat_log1p_test) - 1

users_test_submit = users_test.assign(Calories = yhat_test)[['id', 'Calories']]

In [ ]:
users_test_submit.describe()

In [ ]:
users['calories'].describe()

In [60]:
users_test_submit.to_csv("./data/processed/users_test_ensemble_v3.csv", index=False)